In [1]:
import requests
import warnings
import datetime as dt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib qt5

In [23]:
DATA_SOURCE = "https://data.nsw.gov.au/data/api/3/action/datastore_search?resource_id=2776dbb8-f807-4fb2-b1ed-184a6fc2c8aa&limit=10000"

data = requests.get(DATA_SOURCE)
data = data.json()

if not data["success"]:
    error = data["error"]
    del data
    raise RuntimeError(f"Failed to retrieve data. Error details: {error}.")

if not data["result"]["total_was_estimated"]:
    if data["result"]["limit"] < data["result"]["total"]:
        warnings.warn(f"Incomplete data retrieved. Total dataset size: {data['result']['total']}, retrieved: {data['result']['limit']}.")
else:
    warnings.warn(f"Dataset size unknown, data may be incomplete.")

# Figure out column types
known_fields_types = {
    "postcode": np.uint32,
}
columns = [(d["id"], str) if d["id"] not in known_fields_types else (d["id"], known_fields_types[d["id"]]) for d in data["result"]["fields"]]
columns = dict(columns)

# Convert to pandas
data = pd.DataFrame.from_records(data["result"]["records"])

# Filter out overseas datapoints
#data = data.dropna()
data.postcode[data.postcode == "Masked"] = 1999
data.postcode[data.postcode.isna()] = 1999
data.postcode[data.postcode == "1730"] = 2147 # Seven hills has two postcodes??
data = data.astype(columns)
data.notification_date = pd.to_datetime(data.notification_date)

In [26]:
location_mesh = data[data.notification_date > "2021-07-01"].groupby(["notification_date", "postcode"]).size().unstack(fill_value=0)
plt.pcolormesh(location_mesh.index, location_mesh.columns, np.log10(location_mesh.values.T), shading="nearest")

C:\Users\v-sepuka\AppData\Local\Temp/ipykernel_15404/2914422096.py:2: RuntimeWarning: divide by zero encountered in log10
  plt.pcolormesh(location_mesh.index, location_mesh.columns, np.log10(location_mesh.values.T), shading="nearest")


In [25]:
location_mesh

postcode,1999,2000,2008,2009,2010,2011,2017,2018,2020,2021,...,2761,2762,2763,2765,2766,2767,2768,2770,2777,2866
notification_date,,,,,,,,,,,,,,,,,,,,,
2021-07-02,6,0,0,0,2,1,0,1,2,0,...,0,0,0,0,0,0,1,0,0,0
2021-07-03,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-04,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-05,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-07-06,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2021-07-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2021-07-08,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2021-07-09,2,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2021-07-10,2,4,1,2,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
BOUNDARIES_URL = "https://data.gov.au/geoserver/nsw-suburb-locality-boundaries-psma-administrative-boundaries/wfs?request=GetFeature&typeName=ckan_91e70237_d9d1_4719_a82f_e71b811154c6&outputFormat=json"
data = requests.get(BOUNDARIES_URL)
data.json()